# Data Cleaning

In [1]:
# import libraries
import numpy as np
import pandas as pd

## Step 1: import data and take a quick look

In [2]:
# import data
df = pd.read_csv('OnlineRetail.csv', encoding='ISO-8859-1')

In [3]:
# a quick look
display(df.head())
print('data shape:', df.shape, '\n')
print(df.info())

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/10 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/10 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/10 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/10 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/10 8:26,3.39,17850.0,United Kingdom


data shape: (541909, 8) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
InvoiceNo      541909 non-null object
StockCode      541909 non-null object
Description    540455 non-null object
Quantity       541909 non-null int64
InvoiceDate    541909 non-null object
UnitPrice      541909 non-null float64
CustomerID     406829 non-null float64
Country        541909 non-null object
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB
None


### It looks like CustomerID and Desciption have missing values.

In [4]:
print(df.isnull().sum())

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64


## Step 2: deal with CustomerID
* Option 1:
Keep obs that have missing CustomerID as long as other key info are available, because our forecasting focus is Quantity. This way we retain as much data as possible. 
* Option 2:
Drop obs that have missing CustomerID. Entries with missing CustomerID are more likely to have other mistakes that happened at data entry stage and cannot be detected accurately now.

I chose Option 2. Data looks much messier under Option 1, corroborating my hypothesis in Option 2.  

In [5]:
# drop entires w/ missing CustomerID
df = df.drop(df[df.CustomerID.isnull()].index)
df.shape
# 541909 - 135080 = 406829

(406829, 8)

## Step 3: deal with Quantity
We have negative Quantity values and they're associated with cancelled orders. Based on our forecasting goal, we remove cancelled orders from the dataset. After removing negative Quantity, all cancelled orders are also removed.  

In [6]:
display(df[df.Quantity < 0].head())
display(df[df.InvoiceNo.str.contains("C") == True].shape)
display(df[df.Quantity < 0].shape)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,12/1/10 9:41,27.50,14527.0,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,12/1/10 9:49,4.65,15311.0,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,12/1/10 10:24,1.65,17548.0,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,12/1/10 10:24,0.29,17548.0,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,12/1/10 10:24,0.29,17548.0,United Kingdom


(8905, 8)

(8905, 8)

In [7]:
# drop negative Quantity/cancelled orders
df = df[df.Quantity >= 0]
df.shape
# 406829 - 9805 = 397924

(397924, 8)

## Step 4: deal with Description and StockCode
After the above procedures, we no longer have missing Description values. But there are duplicate Description values associated with StockCode. Thus, I corrected the mapping btw StockCode and Description.

In [8]:
print('Any missing Description?', df['Description'].isnull().sum())
print('Total # of unique StockCode:', df['StockCode'].nunique())
print('Total # of unique Description:', df['Description'].nunique())

Any missing Description? 0
Total # of unique StockCode: 3665
Total # of unique Description: 3877


In [9]:
# confirm duplicated Description 
temp = df.drop_duplicates(['StockCode', 'Description'])
temp = temp.sort_values(by=['StockCode', 'Description'])
dup  = temp[temp.duplicated(['StockCode'])]
display(dup.head())
display(temp[temp.StockCode=='16156L'])

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
47284,540404,16156L,"WRAP, CAROUSEL",25,1/7/11 9:40,0.42,12947.0,United Kingdom
347764,567338,17107D,FLOWER FAIRY 5 SUMMER DRAW LINERS,6,9/19/11 15:00,2.55,14146.0,United Kingdom
6467,536945,17107D,"FLOWER FAIRY,5 SUMMER B'DRAW LINERS",3,12/3/10 12:24,2.55,14083.0,United Kingdom
29444,538799,20622,VIPPASSPORT COVER,1,12/14/10 11:48,2.10,15039.0,United Kingdom
58132,541220,20725,LUNCH BAG RED SPOTTY,200,1/14/11 14:11,1.45,14156.0,EIRE


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
138080,548189,16156L,WRAP CAROUSEL,25,3/29/11 15:15,0.42,16607.0,United Kingdom
47284,540404,16156L,"WRAP, CAROUSEL",25,1/7/11 9:40,0.42,12947.0,United Kingdom


In [10]:
# correct the mapping
# delete duplicate Description
temp = df.drop_duplicates(['StockCode', 'Description'])
temp = temp.sort_values(by=['StockCode', 'Description'])
temp = temp[['StockCode', 'Description']]
temp.drop_duplicates('StockCode', keep='first', inplace=True)

# correct the mapping of StockCode to Description
df = pd.merge(df, temp, on='StockCode', how='left')
df.drop('Description_x', axis=1, inplace=True)
df.rename(columns={'Description_y':'Description'}, inplace=True)

# check
display(df.head())
df.shape

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Description
0,536365,85123A,6,12/1/10 8:26,2.55,17850.0,United Kingdom,CREAM HANGING HEART T-LIGHT HOLDER
1,536365,71053,6,12/1/10 8:26,3.39,17850.0,United Kingdom,WHITE METAL LANTERN
2,536365,84406B,8,12/1/10 8:26,2.75,17850.0,United Kingdom,CREAM CUPID HEARTS COAT HANGER
3,536365,84029G,6,12/1/10 8:26,3.39,17850.0,United Kingdom,KNITTED UNION FLAG HOT WATER BOTTLE
4,536365,84029E,6,12/1/10 8:26,3.39,17850.0,United Kingdom,RED WOOLLY HOTTIE WHITE HEART.


(397924, 8)

Some StockCodes aren't 5-digit integers. 
* For example, POST means postage and C2 means carriage. These entires are removed. 
* Some StockCodes have an alphabet in the end. The alphabet indicates a specific variant of a product. I chose not to group the variants into one big category. Because stores are more likely to want to know the order quantity for each variant. 

In [11]:
display(temp.tail(10))

,StockCode,Description
2424,90214V,"LETTER ""V"" BLING KEY RING"
527067,90214W,"LETTER ""W"" BLING KEY RING"
193006,90214Y,"LETTER ""Y"" BLING KEY RING"
527069,90214Z,"LETTER ""Z"" BLING KEY RING"
4406,BANK CHARGES,Bank Charges
1423,C2,CARRIAGE
317507,DOT,DOTCOM POSTAGE
2239,M,Manual
157195,PADS,PADS TO MATCH ALL CUSHIONS
45,POST,POSTAGE


In [12]:
# drop irrelevant StockCode
df = df.drop(df[df.StockCode.str.len()<5].index)
df = df.drop(df[df.StockCode=='BANK CHARGES'].index)
df.shape

(396370, 8)

## Step 5: deal with Country
Entries with 'Unspecified' and 'European Community' Country values are removed. 

In [13]:
print('Total # of unique countries:', df['Country'].nunique())
print('Unique countries:', df['Country'].unique())

Total # of unique countries: 37
Unique countries: ['United Kingdom' 'France' 'Australia' 'Netherlands' 'Germany' 'Norway'
 'EIRE' 'Switzerland' 'Spain' 'Poland' 'Portugal' 'Italy' 'Belgium'
 'Lithuania' 'Japan' 'Iceland' 'Channel Islands' 'Denmark' 'Cyprus'
 'Sweden' 'Finland' 'Austria' 'Greece' 'Singapore' 'Lebanon'
 'United Arab Emirates' 'Israel' 'Saudi Arabia' 'Czech Republic' 'Canada'
 'Unspecified' 'Brazil' 'USA' 'European Community' 'Bahrain' 'Malta' 'RSA']


In [14]:
df = df.drop(df[df.Country.isin(['Unspecified','European Community'])].index)
df.shape

(396069, 8)

## Step 6: deal with InvoiceDate
Convert InvoiceDate to datetime

In [15]:
# incorrect type for InvoiceDate
print('Data is from', df['InvoiceDate'].min(), 'to', df['InvoiceDate'].max())

Data is from 1/10/11 10:32 to 9/9/11 9:52


In [16]:
# add a datetime variable 'Date' to record the invoice date
timestamp  = list(df.InvoiceDate)
timeframe  = pd.DataFrame(pd.to_datetime(timestamp), columns=['Date'])
df['Date'] = timeframe['Date'].values
display(df)

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Description,Date
0,536365,85123A,6,12/1/10 8:26,2.55,17850.0,United Kingdom,CREAM HANGING HEART T-LIGHT HOLDER,2010-12-01 08:26:00
1,536365,71053,6,12/1/10 8:26,3.39,17850.0,United Kingdom,WHITE METAL LANTERN,2010-12-01 08:26:00
2,536365,84406B,8,12/1/10 8:26,2.75,17850.0,United Kingdom,CREAM CUPID HEARTS COAT HANGER,2010-12-01 08:26:00
3,536365,84029G,6,12/1/10 8:26,3.39,17850.0,United Kingdom,KNITTED UNION FLAG HOT WATER BOTTLE,2010-12-01 08:26:00
4,536365,84029E,6,12/1/10 8:26,3.39,17850.0,United Kingdom,RED WOOLLY HOTTIE WHITE HEART.,2010-12-01 08:26:00
...,...,...,...,...,...,...,...,...,...
397919,581587,22613,12,12/9/11 12:50,0.85,12680.0,France,PACK OF 20 SPACEBOY NAPKINS,2011-12-09 12:50:00
397920,581587,22899,6,12/9/11 12:50,2.10,12680.0,France,CHILDREN'S APRON DOLLY GIRL,2011-12-09 12:50:00
397921,581587,23254,4,12/9/11 12:50,4.15,12680.0,France,CHILDRENS CUTLERY DOLLY GIRL,2011-12-09 12:50:00
397922,581587,23255,4,12/9/11 12:50,4.15,12680.0,France,CHILDRENS CUTLERY CIRCUS PARADE,2011-12-09 12:50:00


In [17]:
# check
print('Data is from', df['Date'].min(), 'to', df['Date'].max())
df = df.drop(columns='InvoiceDate')
df.shape

Data is from 2010-12-01 08:26:00 to 2011-12-09 12:50:00


(396069, 8)

## Step 7: take a look at the cleaned-up dataset
Some obs have 0 as their UnitPrice. I chose to keep them because it might indicate sales promotions.

In [18]:
display(df.describe(include='all'))

,InvoiceNo,StockCode,Quantity,UnitPrice,CustomerID,Country,Description,Date
count,396069,396069,396069.000000,396069.00000,396069.000000,396069,396069,396069
unique,18394,3659,NaN,NaN,NaN,35,3641,17160
top,576339,85123A,NaN,NaN,NaN,United Kingdom,CREAM HANGING HEART T-LIGHT HOLDER,2011-11-14 15:27:00
freq,541,2035,NaN,NaN,NaN,354005,2035,541
first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010-12-01 08:26:00
last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011-12-09 12:50:00
mean,NaN,NaN,13.050635,2.86758,15302.603362,NaN,NaN,NaN
std,NaN,NaN,180.802046,4.26512,1709.768762,NaN,NaN,NaN
min,NaN,NaN,1.000000,0.00000,12346.000000,NaN,NaN,NaN
25%,NaN,NaN,2.000000,1.25000,13975.000000,NaN,NaN,NaN


## Step 8: export the cleaned-up data

In [19]:
df.to_csv('CleanData.csv')